# lib_layered_config Install

In [ ]:
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade setuptools

# install lib_layered_config from GitHub (default)
!{sys.executable} -m pip install --upgrade git+https://github.com/bitranox/lib_layered_config.git

## Compute environment prefixes

The helper `default_env_prefix` converts a slug into the uppercase prefix used for environment variables.

In [ ]:
from lib_layered_config import default_env_prefix

default_env_prefix("demo-config")

## Generate example files Windows

The `examples` module scaffolds commented configuration files for each layer. They are never overwritten unless you pass `force=True`.

In [ ]:
from lib_layered_config.examples.generate import generate_examples
from pathlib import Path
import tempfile

root = Path(tempfile.mkdtemp())
written = generate_examples(root, slug="demo-config", vendor="Acme", app="ConfigKit", platform="windows")
[w.relative_to(root) for w in written]

## Generate example files POSIX

The `examples` module scaffolds commented configuration files for each layer. They are never overwritten unless you pass `force=True`.

In [ ]:
from lib_layered_config.examples.generate import generate_examples
from pathlib import Path
import tempfile

root = Path(tempfile.mkdtemp())
written = generate_examples(root, slug="demo-config", vendor="Acme", app="ConfigKit", platform="posix")
[w.relative_to(root) for w in written]

## Merge all layers

Create temporary application/user files, a `.env`, and an environment variable to demonstrate precedence.

In [ ]:
from lib_layered_config import read_config, default_env_prefix
from pathlib import Path
import tempfile, os

workdir = Path(tempfile.mkdtemp())
etc_root = workdir / "etc" / "demo-config"
user_root = workdir / "xdg" / "demo-config"

etc_root.mkdir(parents=True)
(user_root / "config.d").mkdir(parents=True)

(etc_root / "config.toml").write_text(
    """[service]
endpoint = "https://api.example.com"
""",
    encoding="utf-8",
)
(user_root / "config.toml").write_text(
    """[service]
timeout = 15
""",
    encoding="utf-8",
)
(user_root / ".env").write_text(
    """SERVICE__TIMEOUT=20
""",
    encoding="utf-8",
)

slug = "demo-config"
prefix = default_env_prefix(slug)
os.environ["LIB_LAYERED_CONFIG_ETC"] = str(workdir / "etc")
os.environ["XDG_CONFIG_HOME"] = str(workdir / "xdg")
os.environ[f"{prefix}_SERVICE__TIMEOUT"] = "25"

config = read_config(vendor="Acme", app="ConfigKit", slug=slug, start_dir=str(user_root))
config.as_dict()

# lib_layered_config Demo
Deploys configuration files, reads the merged result, and exercises the CLI.

## Import Modules


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory
import os
import json
from lib_layered_config import read_config, default_env_prefix, deploy_config, i_should_fail
from lib_layered_config.examples import deploy_config as deploy_helper

## Deploy Configfiles


In [ ]:
  from pathlib import Path
  from tempfile import TemporaryDirectory
  import os
  from lib_layered_config import read_config, default_env_prefix, deploy_config, i_should_fail
  from lib_layered_config.examples import deploy_config as deploy_helper

  with TemporaryDirectory() as tmp:
      base = Path(tmp)
      source = base / "base.toml"
      source.write_text('[service]\nendpoint = "https://api.example.com"\n', encoding="utf-8")

      etc_root = base / "etc"
      xdg_root = base / "xdg"

      backup = {k: os.environ.get(k) for k in ("LIB_LAYERED_CONFIG_ETC", "XDG_CONFIG_HOME")}
      os.environ["LIB_LAYERED_CONFIG_ETC"] = str(etc_root)
      os.environ["XDG_CONFIG_HOME"] = str(xdg_root)
      try:
          paths = deploy_helper(source, vendor="Acme", app="Demo", targets=["app", "user"], slug="demo")
          cfg = read_config(vendor="Acme", app="Demo", slug="demo")
          endpoint = cfg.get("service.endpoint")
          print(paths)
          print(endpoint)
          os.environ["LIB_LAYERED_CONFIG_ETC"] = "/etc"
          os.environ["XDG_CONFIG_HOME"] = "~"
          paths = deploy_config(source, vendor="Acme", app="Demo", targets=["user"], slug="demo-alt", force=True)
          cfg = read_config(vendor="Acme", app="Demo", slug="demo")
          print(paths)
          print(str(cfg))
      finally:
          for key, value in backup.items():
              if value is None:
                  os.environ.pop(key, None)
              else:
                  os.environ[key] = value

## Read Config Values via Script as JSON


In [ ]:
!{sys.executable} -m lib_layered_config read --vendor Acme --app "Demo" --slug "demo" --provenance 


## Fail

In [ ]:
!{sys.executable} -m lib_layered_config fail


## Fail with Traceback

In [ ]:
!{sys.executable} -m lib_layered_config --traceback fail
